# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_weather = pd.read_csv("../WeatherPy/city_weather.csv")
city_weather = city_weather[['City', 'Latitude', 'Longitude',
       'Max Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country',
       'Date']]
city_weather.head(5)

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,sitka,57.05,-135.33,53.60,81,75,3.20,US,1598722385
1,faanui,-16.48,-151.75,77.27,81,37,26.22,PF,1598722054
2,thohoyandou,-22.95,30.48,81.90,19,15,3.62,ZA,1598722562
3,bubaque,11.28,-15.83,78.80,86,100,18.79,GW,1598722383
4,puerto ayora,-0.74,-90.35,73.40,69,20,19.46,EC,1598722347


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
lat = city_weather["Latitude"]
lon = city_weather["Longitude"]


#Customize the size of the figure

figure_layout = {
    'width': '750px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#Add heatmap layer
locations = city_weather[["Latitude", "Longitude"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Filter for temperatures between 60 & 90 degrees fahrenheit, humidity less than 40%, and wind speed under 10 MPH
narrowed_cities = city_weather.loc[(city_weather["Max Temperature"] >= 75) & (city_weather["Max Temperature"] <= 90) & (city_weather["Humidity"] < 40) 
                                   & (city_weather["Wind Speed"] < 10) ,:]

#Print preview of new dataframe
narrowed_cities.head(5)

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
2,thohoyandou,-22.95,30.48,81.90,19,15,3.62,ZA,1598722562
213,serowe,-22.38,26.72,75.42,24,82,6.55,BW,1598722595
243,blackfoot,43.19,-112.34,79.00,21,41,7.00,US,1598722600
256,sanandaj,35.31,46.99,78.80,14,0,2.24,IR,1598722603
317,nokaneng,-19.67,22.27,75.72,19,11,8.39,BW,1598722614


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = narrowed_cities[['City', 'Latitude', 'Longitude', 'Max Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
hotel_df = hotel_df[['City', 'Latitude', 'Longitude', 'Max Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
#hotel_df = hotel_df.loc[(hotel_df["City"] != "abu samrah") & (hotel_df["City"] != "san borja") & 
                        #(hotel_df["City"] != "baijiantan") ,:]
hotel_df = hotel_df.reset_index()
hotel_df.head(15)

,index,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,2,thohoyandou,-22.95,30.48,81.90,19,15,3.62,ZA,1598722562
1,213,serowe,-22.38,26.72,75.42,24,82,6.55,BW,1598722595
2,243,blackfoot,43.19,-112.34,79.00,21,41,7.00,US,1598722600
3,256,sanandaj,35.31,46.99,78.80,14,0,2.24,IR,1598722603
4,317,nokaneng,-19.67,22.27,75.72,19,11,8.39,BW,1598722614
5,337,edirne,41.68,26.56,78.01,37,0,1.99,TR,1598722617
6,339,general cepeda,25.38,-101.45,82.40,39,5,7.20,MX,1598722618
7,542,duiwelskloof,-23.69,30.14,79.93,20,67,4.23,ZA,1598722656


In [10]:
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = { "key": g_key,
    "radius": 5000,
    "type": "lodging"
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    city = row["City"]
    # get lat, lng from df
    hotel_lat = row["Latitude"]
    hotel_lng = row["Longitude"]
#     # change location each iteration while leaving original params in place
    params["location"] = f"{hotel_lat}, {hotel_lng}"
    print(f"Retrieving results for {index}: {city}")
#     # Query url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#     # make request and print url
    response = requests.get(base_url, params=params).json()
    
# #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

#     # convert to json
    results = response['results']
#----------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------
# # #     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



Retrieving results for 0: thohoyandou
Retrieving results for 1: serowe
Retrieving results for 2: blackfoot
Retrieving results for 3: sanandaj
Retrieving results for 4: nokaneng
Missing field/result... skipping.
Retrieving results for 5: edirne
Retrieving results for 6: general cepeda
Retrieving results for 7: duiwelskloof


In [11]:
hotel_df

,index,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,2,thohoyandou,-22.95,30.48,81.90,19,15,3.62,ZA,1598722562,Masili Guesthouse & Conference
1,213,serowe,-22.38,26.72,75.42,24,82,6.55,BW,1598722595,Hill View Inn
2,243,blackfoot,43.19,-112.34,79.00,21,41,7.00,US,1598722600,Super 8 by Wyndham Blackfoot
3,256,sanandaj,35.31,46.99,78.80,14,0,2.24,IR,1598722603,Sanandaj Tourism Hotel
4,317,nokaneng,-19.67,22.27,75.72,19,11,8.39,BW,1598722614,
5,337,edirne,41.68,26.56,78.01,37,0,1.99,TR,1598722617,Rys Hotel & Restaurant
6,339,general cepeda,25.38,-101.45,82.40,39,5,7.20,MX,1598722618,Hacienda Los Jose
7,542,duiwelskloof,-23.69,30.14,79.93,20,67,4.23,ZA,1598722656,hoshani security services


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
lat = city_weather["Latitude"]
lon = city_weather["Longitude"]

coordinates = hotel_df[["Latitude", "Longitude"]]

#Customize the size of the figure

figure_layout = {
    'width': '750px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)

#Add heatmap layer
locations = city_weather[["Latitude", "Longitude"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…